In [5]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps


# import AnimalShelter class from the animal_shelter CRUD module
from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# Hardcode username and password infomration and instantiate animalshelter object from CRUD Python module 
username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))
#preload the filtered options globally to ensure that these expensive operations are done when app loads 
#and not done in a callback where issues may occur
waterBreeds = ["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]
dfWater = pd.DataFrame.from_records(shelter.read_filtered("Dog", waterBreeds, "Intact Female", 26, 156))
mwBreeds = ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]
dfMW = pd.DataFrame.from_records(shelter.read_filtered("Dog", mwBreeds, "Intact Male", 26, 156))
ditBreeds = ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]
dfDIT = pd.DataFrame.from_records(shelter.read_filtered("Dog", ditBreeds, "Intact Male", 20, 300))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Project2')

#Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#Place the HTML image tag in the line below into the app.layout code according to your design
#include a unique identifier such as your name or date

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A(
        href="https://www.snhu.edu",
        children= [
            html.Center(html.Img(
                id='gsImage',
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                alt="Grazioso Salvare's Logo")),
        ]),                 
    html.Center(html.B(html.H1('SNHU CS-340 Grazioso Salvare Dashboard'))),
    html.Center(html.B(html.H4('Joseph Kim Project 2'))),
    html.Hr(),
#Added radio buttons for the interactive filtering options. 
    html.Div([html.Center(
        html.H6("Filter by Rescue type: ")),
        html.Center(dcc.RadioItems(
            id = 'radio-items', 
            options = [
                {'label' : 'Water Rescue', 'value' : 'WR'},
                {'label' : 'Mountain/Wilderness Rescue', 'value' : 'MWR'},
                {'label' : 'Disaster Rescue/Individual Tracking', 'value' : 'DRIT'},
                {'label' : 'No Filter', 'value' :'No'}
            ],
            value = 'No', #sets filter to none to start
            labelStyle = {'display' : 'inline-block'}
        ))      
    ]),
    html.Hr(),
    html.Center(html.H4("Austin Animal Care Table of Animals")),
    #create a data table using dash_table 
    dt.DataTable(
        id='datatable-interactivity',
        #for each column in df, set the name and id and set column options deletable and selectable 
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        #use data from df to populate the data table. Sends data from df to the data table
        data=df.to_dict('records'),
        #Set up the features for your interactive data table to make it user-friendly for your client
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi", 
        row_selectable = "single", 
        row_deletable = False,
        selected_columns = [], 
        selected_rows = [], 
        page_action = "native",
        page_current = 0,
        page_size = 10,
        
        style_data = {
            #"width": "80px", "minWidth": "30px", "maxWidth":"100px",
            "width": "5%", "minWidth": "5%px", "maxWidth":"10%",
            "overflow": "hidden",
            "textOverflow": "ellipsis",
        }
        
    ),
    html.Br(),
    html.Hr(),
#This div sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(style = {"display" : "flex"}, children=[
        html.H4("Geolocation map showing the location of the animal from the table",
                style = {"display":"inline-block", "margin-left": "45%"})
    ]),
    html.Div(className='row',
         style={'display' : 'flex'}, children=[
            #this div will be where the pie chart is displayed
            html.Div(
                className='col s12 m6',
                style = {"display": "flex"},
                children =[
                    dcc.Graph(
                        id = "graph-id",
                    ),    
               #this div will be where the geolocation map is displayed
                html.Div(
                    id = "map-id",
                    className='col s12 m6',
                    style = {"display": "flex"},
                )
                ]
            ),
        
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
  
###callback for interactive data table based on radio button values    
@app.callback([Output('datatable-interactivity','data'),
               Output('datatable-interactivity','columns')],
              [Input('radio-items', 'value')])
def update_dashboard(filter_type):
### callback filters interactive data table with dataframes created with MongoDB queries
    if filter_type == "WR":
        #loads the dataframe with water rescue filter information 
        dff = dfWater
        #queries the dataframe to set the min and max age as the find() with min and max age not working...
        #dff = dff.query('26 <= age_upon_outcome_in_weeks <= 156')
        
        """
        Logic to filter the dataframe using pandas functionality without having to load multiple data frames
        Would use this logic but I believe grading rubric requires use of a filtering CRUD method
        dff = df.loc[df["animal_type"] == "Dog"]
        dff = dff.loc[(dff["breed"] == "Labrador Retriever Mix") | 
                     (dff["breed"] == "Chesapeake Bay Retriever") |
                     (dff["breed"] == "Newfoundland")
                     ]
        dff = dff.loc[(dff['sex_upon_outcome'] == 'Intact Female')]
        dff = dff.query('26 <= age_upon_outcome_in_weeks <= 156')
        """
    elif filter_type == "MWR":
        #loads the dataframe with mountain or wilderness rescue filter information
        dff = dfMW
        
        """
        Logic to filter the dataframe using pandas functionality without having to load multiple data frames
        Would use this logic but I believe grading rubric requires use of a filtering CRUD method
        dff = pd.DataFrame.from_records(shelter.read({'animal_type': 'Dog'}))
        dff = dff.loc[(dff["breed"] == "German Shepherd") |
                      (dff["breed"] == "Alaskan Malamute") |
                      (dff["breed"] == "Old English Sheepdog") |
                      (dff["breed"] == "Siberian Husky") |
                      (dff["breed"] == "Rottweiler")]
        dff = dff.loc[(dff["sex_upon_outcome"] == "Intact Male")]
        dff = dff.query("26 <= age_upon_outcome_in_weeks <= 156")
        """
    elif filter_type == "DRIT":
        #loads the dataframe with disaster or individual tracking filter information
        dff = dfDIT
        """
        Logic to filter the dataframe using pandas functionality without having to load multiple data frames
        Would use this logic but I believe grading rubric requires use of a filtering CRUD method
        dff = pd.DataFrame.from_records(shelter.read({'animal_type': 'Dog'}))
        dff = dff.loc[(dff["breed"] == "German Shepherd") |
                      (dff["breed"] == "Doberman Pinscher") |
                      (dff["breed"] == "Golden Retriever") |
                      (dff["breed"] == "Bloodhound") |
                      (dff["breed"] == "Rottweiler")]
        dff = dff.loc[(dff["sex_upon_outcome"] == "Intact Male")]
        dff = dff.query("20 <= age_upon_outcome_in_weeks <= 300")
        """
    else:
         dff = df #pd.DataFrame.from_records(shelter.read({}))  
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in dff.columns]
    data=dff.to_dict('records')
        
    return (data,columns)



### callback function to highlight selected cell 
@app.callback(
    Output('datatable-interactivity', 'style_data_conditional'),
    [Input('datatable-interactivity', 'selected_columns')]
)
def update_styles(selected_columns):
    """highlight the selected cell of dash table"""
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


###callback function to create a pie chart and return the chart to the graph-id ID in app.layout
@app.callback(
    Output('graph-id', "figure"),
    [Input('datatable-interactivity', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame(viewData)
    pieChart = px.pie(
        data_frame = dff,
        names = dff["breed"],
        hole = 0.3,
        title = "Pie chart showing the distribution of breeds"
        
    )
    return pieChart

###callback function to create and update the geolocation map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-interactivity', "derived_viewport_data"),
     Input('datatable-interactivity', 'derived_virtual_selected_rows')
     ])
def update_map(viewData, derived_virtual_selected_rows):
     #functionality for updating map. map always shows location of the animal at top of table's current page 
    
    dff = pd.DataFrame.from_dict(viewData)

    # if there are no selected rows yet, map default displays first animal of table's current page
    if not derived_virtual_selected_rows:
        latitude = dff.iloc[0,13]
        longitude = dff.iloc[0,14]
        animal_breed = dff.iloc[0,4]
        animal_name = dff.iloc[0,9]
    # else there is a selected row, map displays that animal
    else:
        selected_int = derived_virtual_selected_rows[0]
        latitude = dff.iloc[selected_int, 13]
        longitude = dff.iloc[selected_int, 14]
        animal_breed = dff.iloc[selected_int,4]
        animal_name = dff.iloc[selected_int,9]


    return [
        dl.Map(style={'width': '700px', 'height': '500px'}, 
               center=[latitude, longitude], 
               zoom=10, 
               children=[ 
                    dl.TileLayer(id="base-layer-id"),
                    # Marker with tool tip and popup
                    dl.Marker(position=[latitude, longitude], 
                        children=[
                        # show breed of animal on hovering over marker
                        dl.Tooltip(animal_breed),
                        # show animal name on clicking marker
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(animal_name)
                        ])
                    ]),
                   
        ])
    ]


app